# AI 201 Programming Assignment 1
## A* Algorithm Implementation

Submitted by: 
Jan Lendl R. Uy, 2019-00312

### Notebook Prerequisites
Install the following Python modules, if not yet installed:
1. numpy

In [1]:
import copy

In [2]:
# CONSTANTS
INPUT_FILE_PATH = "astar_in.txt"
MAX_ALGORITHM_ITERATIONS = 10

In [3]:
def read_file(path):
    
    with open(path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store the start state and goal state
    start_state = []
    goal_state = []

    # Initialize a flag to keep track if following tiles belong to the start
    # or the goal state
    reading_part = None

    for line in lines:
        line = line.strip() # Remove leading/trailing whitespace
        if line == "start":
            reading_part = "start"
        elif line == "goal":
            reading_part = "goal"
        elif line and reading_part:
            # Convert line into a list, handling '*' and converting numbers to integers
            line_list = [int(x) if x.isdigit() else x for x in line.split()]
            if reading_part == "start":
                start_state.append(line_list)
            elif reading_part == "goal":
                goal_state.append(line_list)
                
    start_state = tuple(tuple(row) for row in start_state)
    goal_state = tuple(tuple(row) for row in goal_state)

    return start_state, goal_state

start, goal = read_file(INPUT_FILE_PATH)
print(f"Start State: {start}")
print(f"Goal State: {goal}")

Start State: (('*', 1, 3), (8, 2, 4), (7, 6, 5))
Goal State: ((1, 2, 3), (8, '*', 4), (7, 6, 5))


In [4]:
class PuzzleBoard:
    
    def __init__(self, board_as_2d_tuple):
        self.board_contents = board_as_2d_tuple
        self.blank_tile_coords = self.get_blank_tile_coords()
        self.valid_movements = self.get_valid_movements()   
        
        self.board_in_str = ""
        for row in self.board_contents:
            # print(f"row = {row}")
            for tile in row:
                # print(f"tile = {tile}")
                self.board_in_str += f"| {tile} "
            self.board_in_str += "|\n"
       
    # Print override
    # Formats the Puzzle printing as a 3x3 grid-like structure 
    def __str__(self):
        return self.board_in_str
    
    # Equality override
    # Checks if two boards have exactly the same elements in 
    # the same location 
    def __eq__(self, other_puzzle):
        return isinstance(other_puzzle, PuzzleBoard) and self.board_contents == other_puzzle.board_contents
    
    def __hash__(self):
        return hash(tuple(self.board_contents))
    
    def get_board_as_string(self):
        return self.board_in_str 
    
    def add_coordinates(self, coords_1, coords_2):
        return tuple(map(lambda a, b: a + b, coords_1, coords_2))    
    
    def swap(self, coords_orig, coords_new):
        x, y = coords_orig
        new_x, new_y = coords_new
        board_copy = list(list(row) for row in self.board_contents)
        board_copy[x][y], board_copy[new_x][new_y] = board_copy[new_x][new_y], board_copy[x][y]
        return tuple(tuple(row) for row in board_copy)
        
    def get_valid_movements(self):
        # Define possible movements: up, down, left, right
        movements = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}
        
        # Define a dictionary to store valid movements and the corresponding 
        # indices of swaps
        self.valid_movements = {}
        
        # Get valid movements
        for movement in movements:
            check_valid_movement = self.add_coordinates(self.blank_tile_coords, movements[movement])
            x_moved, y_moved = check_valid_movement
            # new_coords.append(check_valid_movement)
            if (x_moved >= 0 and x_moved < 3) and (y_moved >= 0 and y_moved < 3):
                self.valid_movements[movement] = check_valid_movement
            
        return self.valid_movements     
    
    def get_blank_tile_coords(self):
        blank_tile = None
        for row in self.board_contents:
            for tile in row:
                if tile == "*":
                    blank_tile = (self.board_contents.index(row), row.index(tile))
        return blank_tile

starting_puzzle = PuzzleBoard(start)
goal_puzzle = PuzzleBoard(goal)
print(f"Start State: \n{starting_puzzle}")
print(f"Goal State: \n{goal_puzzle}")

Start State: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Goal State: 
| 1 | 2 | 3 |
| 8 | * | 4 |
| 7 | 6 | 5 |



In [5]:
class Node:
    def __init__(self, value, parent=None, g=0, h=0):
        self.value = value
        self.parent = parent
        self.g = g
        self.h = h
        self.f = g + h
    
    def __str__(self):
        return self.value.get_board_as_string()
    
    def __hash__(self):
        return hash(self.value)
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.value == other.value
    
    def increment_path_cost(self):
        self.g += 1

    def path(self):
        node, pth = self, []
        while node:
            pth.append(node.value)
            node = node.parent
        return pth[::-1]

In [6]:
def a_star(start, goal, heuristic=None):
    
    print("Running A* Algorithm")
    print(f"Start Node: \n{start}")
    print(f"Goal Node: \n{goal}")
    
    # Variables to be used throughout the algorithm
    open = set()
    closed = set()
    path_to_goal = None # Store sequence of nodes leading to goal node
    ctr = 0 # Counter to break non-converging search
    f = 0 # Total path cost
    g = 0
                    
    # print(f"Blank tile coordinates: {start.blank_tile_coords}")
    start_node = Node(start, h=number_of_tiles_in_wrong_position(start.board_contents, goal.board_contents))
    goal_node = Node(goal)
        
    # Push starting node to open
    open.add(start_node)
    current_node = start_node
    
    while ctr < MAX_ALGORITHM_ITERATIONS:
        
        print(f"\nStep {ctr+1}")
            
        # If open is empty, return failure and end search
        # Else, continue search
        if len(open) == 0:
            print("No solution found.")
            break
        
        # Remove from OPEN the node whose f value is smallest and put it on a list called closed. 
        # Save this node as the value of current_node
        node_lowest_f = min(open, key=lambda node: node.f)
        current_node = node_lowest_f
        open.remove(node_lowest_f)
        closed.add(node_lowest_f)
        print(f"Current node: \n{current_node}")
        # print(f"node_lowest_f = {node_lowest_f}")
        # print(f"open = {open}")
        # print(f"closed = {closed}")
        
        # If current node is the goal node, return success
        # End search
        if (current_node.value == goal_node.value):
            print(f"Goal achieved!")
            # TODO: Return node sequence for path_to_goal
            path_to_goal = current_node.path()
            break
        
        # Expand current node then add these nodes to the list of
        # opened nodes
        successor_nodes = get_successors(current_node, goal_node)
        if len(successor_nodes) == 0:
            continue
        
        # Put the successor nodes to the list of opened nodes
        # Print opened nodes for checking
        print(f"Opened Nodes:")
        for successor_node in successor_nodes:
            print(f"Successor node: \n{successor_node} \
                        \nf(s) = {successor_node.f} \
                        \ng(s) = {successor_node.g} \
                        \nh(s) = {successor_node.h}")
            open.add(successor_node)
                    
        ctr += 1

    return path_to_goal
    
def get_successors(current, goal):
    
    successors = []
    current_board = current.value
    goal_board = goal.value
        
    # Move the tile according to the specified direction
    for movement in current_board.valid_movements:
        successor = current_board.swap(current_board.blank_tile_coords, current_board.valid_movements[movement])
        successor_puzzle = PuzzleBoard(successor)
        successor_node = Node(successor_puzzle,
                              parent = current,
                              g = current.g+1, 
                              h = number_of_tiles_in_wrong_position(current_board.board_contents, goal_board.board_contents))
        successors.append(successor_node)
        
    return successors

def number_of_tiles_in_wrong_position(current_board, goal_board):
    wrong_counter = 0
    
    for i in range(len(current_board)):
        for j in range(len(current_board[i])):
            if current_board[i][j] != goal_board[i][j]:
                wrong_counter += 1
    return wrong_counter

path = a_star(starting_puzzle, goal_puzzle)
if path:
    print(f"Path from start to goal:")
    for node in path:
        print(node)
else:
    print("No solution found.")

Running A* Algorithm
Start Node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Goal Node: 
| 1 | 2 | 3 |
| 8 | * | 4 |
| 7 | 6 | 5 |


Step 1
Current node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 4                         
g(s) = 1                         
h(s) = 3
Successor node: 
| 1 | * | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 4                         
g(s) = 1                         
h(s) = 3

Step 2
Current node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 6                         
g(s) = 2                         
h(s) = 4
Successor node: 
| 8 | 1 | 3 |
| 7 | 2 | 4 |
| * | 6 | 5 |
                         
f(s) = 6                         
g(s) = 2                         
h(s) = 4
Successor node: 
| 8 | 1 | 3 |
| 2 | * | 4 |
| 7 | 6 | 5 |
 